In [2]:
import torch

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

torch.set_default_device(device)
print(f"Using Device = {torch.get_default_device()}")

Using Device = cpu


In [3]:
import pandas as pd

totalcattle = pd.read_csv("Data/TotalCattle.csv")
janind = totalcattle['Period']=="FIRST OF JAN"
JanData = totalcattle[['Year','Value']][janind].sort_index()
CattleData = JanData['Value'].str.replace(',','').astype(int)

In [6]:
CattleData.shape

(159,)

In [4]:
from torch.utils.data import Dataset
import numpy as np

class CattleDataset(Dataset):
    def __init__(self, data, seq_length=5, pred_length=1):
        self.data = data
        self.seq_length = seq_length
        self.pred_length = pred_length

        self.mean = np.mean(data)
        self.std = np.std(data)
        self.data_norm = (data - self.mean) / self.std
    
    def __len__(self):
        return len(self.data) - self.seq_length - self.pred_length + 1
    
    def __getitem__(self, idx):
        x = self.data_norm[idx:idx + self.seq_length]
        y = self.data_norm[idx + self.seq_length:idx + self.seq_length + self.pred_length]

        x_tensor = torch.FloatTensor(x).unsqueeze(-1)
        y_tensor = torch.FloatTensor(y)

        return x_tensor, y_tensor


In [ ]:
# train_size = int(len(CattleData) * 0.8)
# train_data = CattleData[:train_size]
# test_data = CattleData[train_size:]
from torch.utils.data import DataLoader

train_size = int(len(CattleData) * 0.8)
train_data = CattleData[:train_size]
test_data = CattleData[train_size:]

seq_length = 20
pred_length = 1

train_set = CattleDataset(train_data,seq_length,pred_length)
test_set = CattleDataset(test_data,seq_length,pred_length)

train_dataloader = DataLoader(train_set, batch_size=10, shuffle=True)
test_dataloader  = DataLoader(test_set,  batch_size=10, shuffle=False)


In [6]:
import torch.nn as nn

# class CattleLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(CattleLSTM, self).__init__()

#         self.lstm = nn.LSTM(input_size,hidden_size,batch_first=True)
#         self.h2o = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
    
#     def forward(self,line_tensor):
#         lstm_out, hidden = self.lstm(line_tensor)
#         output = self.h2o(hidden[0][-1])
#         output = self.softmax(output)

#         return output

class CattleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        _, (h_n, _) = self.lstm(x)

        # last layer hidden state
        h_last = h_n[-1]           # (batch, hidden_size)
        out = self.fc(h_last)      # (batch, output_size)

        return out


In [ ]:
n_hidden = 20

lstm = CattleLSTM(input_size=1,hidden_size=n_hidden,output_size=pred_length)

In [8]:
import time

learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # all_losses = []
    model.train()


    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # all_losses.append(loss.item())

        if batch % 5 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"Loss: {loss:>7f}    [{current:<5d}/{size:>5d}]")
    
    # return all_losses
      
    


In [9]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    
    test_loss /= num_batches
    print(f"Test Error: \n Avg Loss: {test_loss:>8f}")
            

In [10]:
X, y = next(iter(train_dataloader))
print(X.shape, y.shape)

ValueError: could not determine the shape of object type 'Series'

In [10]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, lstm, loss_fn, optimizer)
    test_loop(test_dataloader, lstm, loss_fn)
print("Done!")

Epoch 1
-------------------------------


KeyError: 46